# 3. Crawling
페이지 하나만 분석하는 것은 스크레이퍼라고 부르고 여러 페이지, 여러 사이트를 이동하는 스크레이퍼를 크롤러라고 부른다.
웹크롤러라는 이름은 이름 그대로 웹을 크롤링한다는 의미.. 특정 URL에서 페이지를 가져오고, 그 페이지를 검사해서 다른 URL을 찾고, 다시 그 페이지를 가져오는 작업을 반복한다.

# 3.1 단일 도메인 내의 이동
임의의 위키백과 페이지를 가져와서 페이지의 링크 목록을 가져오는 코드를 보자

In [ ]:
from urllib2 import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html, "html.parser")

for link in bsObj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

이 링크에는 '항목링크'와 '항목링크가 아닌 다른 링크'들이 섞여있다.
항목링크가 다른 링크들과 구분되고 항목링크들만이 가진 공통점으로 아래 세가지가 있다.
 - 이 링크들은 id가 bodyContent인 div안에 있다.
 - URL에는 세미콜론이 포함되어있지 않다.
 - URL은 /wiki/로 시작한다.

In [ ]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen("https://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html, "html.parser")

for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
    if 'href' in link.attrs:
        print(link.attrs['href'])

아래 예제는 "https://en.wikipedia.org/wiki/Kevin_Bacon" 페이지의 항목링크들을 저장한 뒤에 각 링크들의 항목링크들을 출력하는 예제이다.

In [ ]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

links = getLinks("/wiki/Kevin_Bacon")
print('@@@@ links type: {0}'.format(type(links)))
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

# 3.2 전체 사이트 크롤링

같은 페이지를 두번 크롤링하지 않으려면 발견된 내부링크가 모두 일정한 형식을 취하고,
프로그램이 동작하는 동안 계속 유지되는 리스트에 보관하는 것이 중요하다.

페이지들을 실제 폴더구조와 같이 정리하고 다시 설계해야하는 페이지가 얼마나 되고 이동해야할 컨텐츠가 얼마나 되는지 산출하는 작업도 중요하다.

이런 작업을 잘하기 위해 필요한 것은 우선 사이트의 페이지 몇개를 살펴보며 패턴을 찾는 것이다.
위키백과에서 항목페이지와 개인정보 정책 페이지 같은 항목 외 페이지를 여럿 보면 아래 같은 패턴이 있다.
 - 항목, 편집 페이지등 페이지들의 제목은 항상 h1태그 안에 있고 h1태그는 페이지당 하나만 존재한다.
 - 모든 바디 텍스트는 div#bodyContent태그에 들어있다. div#mw-content-text -> p로 첫번째 문단 태그만 선택하는 것이 나을 수도 있다.
 - 편집 링크는 항목 페이지에만 존재한다. 존재하면, li#ca-edit -> span ->a로 찾을 수 있다.

페이지를 이동하며 페이지 제목, 첫번째 문단, 편집 페이지를 가리키는 링크를 재귀적으로 수집하는 코드를 만들어보자

In [ ]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id="mw-content-text").findAll("p")[0])
        print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href'])
    except AttributeError:
        print('This page is missing something! No worries though!')
        
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print("--------------\n"+newPage)
                pages.add(newPage)
                getLinks(newPage)
                
getLinks("")

여기서는 타이틀, 첫번째 문단, 편집페이지 링크만 출력했을 뿐, 긁어온 데이터를 저장하지는 않았다.
데이터를 저장하고 데이터베이스를 만드는 것은 추후에 살펴보도록 하자.